In [1]:
import pandas as pd
import numpy as np
import pickle as p

In [2]:
data_all = pd.read_csv('data_all.csv')

# Test with Subset of Data
np.random.seed(41)
item_subset = list(np.random.choice(data_all['Item_ID'].unique(), size=5, replace=False))
data_subset = data_all.loc[data_all['Item_ID'].isin(item_subset)].copy()

# # Test with All Data
# data_subset = data_all.copy()

In [3]:
item_subset

[31, 2259, 1203, 5073, 102]

### `/detect_conflict`

In [3]:
import requests
import json

url = 'http://localhost:5000/detect_conflict/'
with open('constraints_1.json') as f:
    constraints = json.load(f)

payload = {'constraints': constraints
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [4]:
r.json()['conflict']

'No conflict'

### `/train`

In [52]:
import requests
import json

url = 'http://localhost:5000/train/'

data = data_subset.to_json()

payload = {'data': data,
           'cv_acc': True,
           'project_id': 1
           }

# headers = {'content-type': 'application/json',
#            'Accept-Charset': 'UTF-8'
#           }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [59]:
data_subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 126405 entries, 1066 to 3867836
Data columns (total 7 columns):
Wk         126405 non-null int64
Tier       126405 non-null int64
Groups     126405 non-null int64
Store      126405 non-null int64
Item_ID    126405 non-null int64
Qty_       126379 non-null float64
Price_     126379 non-null float64
dtypes: float64(2), int64(5)
memory usage: 12.7 MB


In [53]:
r.json()['result']

'Success'

In [54]:
# CV Accuracies Returned
cv_accuracies = pd.DataFrame.from_dict((r.json()['cv_acc']))
cv_accuracies

,avg_sales,item_id,mae_score,mpe_score,r2_score,rmse_score
0,45.216946,31.0,15.211045,0.336401,0.514597,23.211137
1,176.605667,102.0,44.631816,0.252720,0.562941,61.375934
2,198.348404,1203.0,40.303816,0.203197,0.635932,59.186954
3,97.153709,2259.0,27.536322,0.283430,0.543398,35.557477
4,115.366066,5073.0,28.738143,0.249104,0.480752,37.210351


### `/optimize`

In [5]:
with open('constraints_2.json') as f:
    constraints = json.load(f)

In [ ]:
import requests
import json

url = 'http://localhost:5000/optimize/'
    
payload = {'project_id': 1,
           'constraints': constraints, # two parts, first part constraint, second part price range limits
           'population': 300,
           'max_epoch': 100,
           'price_info_path':'price_info.pkl',
           'model_path': 'models/'
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [14]:
best_prices = r.json()['result']
best_prices

[4.9,
 9.6,
 5.1,
 9.7,
 4.45,
 9.35,
 2.6,
 5.3,
 9.85,
 5.9,
 10.6,
 6.0,
 10.6,
 2.15,
 5.35,
 10.25,
 5.6,
 9.85,
 2.75,
 2.7,
 3.3,
 3.9,
 4.25,
 4.95,
 3.55,
 3.35,
 3.9,
 4.2,
 3.3,
 2.75,
 3.6,
 4.45,
 5.2,
 3.95,
 4.35,
 4.95,
 3.25,
 8.75,
 11.95,
 12.65,
 13.35,
 9.35,
 13.0,
 13.7,
 2.8,
 3.65,
 5.25,
 6.6,
 6.8,
 8.7,
 6.5,
 5.45,
 6.75,
 11.1,
 5.95,
 13.6,
 8.2,
 10.45,
 5.9,
 8.25,
 2.4,
 3.0,
 3.4,
 6.95,
 6.6,
 7.3,
 8.0,
 10.8,
 11.5,
 12.2,
 10.85,
 11.55,
 12.25,
 11.85,
 12.55,
 13.25,
 8.9,
 9.6,
 10.3,
 11.25,
 11.95,
 10.3,
 11.0,
 11.7,
 9.45,
 10.15,
 10.85,
 9.5,
 10.2,
 10.9,
 11.15,
 11.85,
 12.55,
 7.05,
 11.75,
 12.45,
 8.5,
 13.0,
 13.7,
 0.0,
 0.0,
 0.0,
 8.7,
 14.3,
 8.25,
 12.15,
 12.85,
 13.55,
 5.3,
 9.7,
 10.4,
 3.8,
 7.0,
 3.5,
 3.5,
 4.2,
 0.7,
 1.2,
 13.5,
 4.75,
 2.45,
 3.4,
 4.0,
 4.85,
 5.35,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 7.45,
 11.25,
 11.95,
 12.65,


### `/predict`

In [ ]:
import requests
import json

url = 'http://localhost:5000/predict/'
payload = {'prices': best_prices,
           'project_id': 1
           }
headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }
payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

# What If Prediction
qty = r.json()['qty_estimates']

In [ ]:
qty

In [ ]:
# Calculate Revenue
revenue = 0
for item in best_prices.keys():
    revenue += (best_prices[item]*qty['Qty_'+item])
    
print(revenue)

### `/predict/`

In [60]:
# Sample Actual Price 
from helper_functions import *

sales_data = data_subset

sales_data = optimize_memory(sales_data)

sales_data_wide = sales_data.set_index(
    ['Wk', 'Tier', 'Store', 'Item_ID']).unstack(level=-1).reset_index().copy()
sales_data_wide.columns = [
    ''.join(str(i) for i in col).strip()
    for col in sales_data_wide.columns.values
]
sales_data_wide = sales_data_wide.sort_values(
    ['Tier', 'Store', 'Wk'], ascending=True).reset_index(drop=True)

sales_data_wide_clean = sales_data_wide.dropna(axis=0).copy()
dataset = sales_data_wide_clean
price_columns = [
    col for col in sales_data_wide_clean.columns if col.startswith('Price')
]

a = dataset.sample(1)
a_dict = a[price_columns].iloc[0].to_dict()
a_input = {}
for key in a_dict.keys():
    a_input[key.split('_')[1]]=round(a_dict[key],2)

print('Input Prices Quantities:')
a_input

Input Prices Quantities:


{'31': 1.85, '102': 9.65, '1203': 3.2, '2259': 13.0, '5073': 3.7}

In [61]:
# Predict Sales Qty and Compare to Actual

import requests
import json

url = 'http://localhost:5000/predict/'

payload = {'prices': a_input, 'project_id': 1}

headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)
qty = r.json()['qty_estimates']
qty_outp = {}
for key in qty.keys():
    qty_outp[key] = round(qty[key], 0)
actual = a[[
    col for col in sales_data_wide_clean.columns if col.startswith('Qty_')
]].iloc[0].to_dict()
pd.DataFrame([qty_outp, actual], index=['Estimated', 'Actual']).transpose()

,Estimated,Actual
Qty_102,160.0,70.0
Qty_1203,191.0,138.0
Qty_2259,80.0,38.0
Qty_31,40.0,44.0
Qty_5073,91.0,26.0
